In [1]:
# Install scikit-optimize
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [scikit-optimize]


In [2]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from skopt import gp_minimize
from skopt.space import Integer, Categorical
from skopt.utils import use_named_args
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Load the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = [
    'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
    'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'
]
df = pd.read_csv(url, header=None, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
# Data Preprocessing
missing_columns = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for column in missing_columns:
    df[column].replace(0, np.nan, inplace=True)

df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,NaN,33.6,0.627,50,1
1,1,85.0,66.0,29.0,NaN,26.6,0.351,31,0
2,8,183.0,64.0,NaN,NaN,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0
764,2,122.0,70.0,27.0,NaN,36.8,0.340,27,0
765,5,121.0,72.0,23.0,112.0,26.2,0.245,30,0
766,1,126.0,60.0,NaN,NaN,30.1,0.349,47,1


In [5]:
df.fillna(df.median(), inplace=True)
X = df.drop('Outcome', axis=1)
y = df['Outcome']
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148.0,72.0,35.0,125.0,33.6,0.627,50
1,1,85.0,66.0,29.0,125.0,26.6,0.351,31
2,8,183.0,64.0,29.0,125.0,23.3,0.672,32
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63
764,2,122.0,70.0,27.0,125.0,36.8,0.340,27
765,5,121.0,72.0,23.0,112.0,26.2,0.245,30
766,1,126.0,60.0,29.0,125.0,30.1,0.349,47


In [6]:
y

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

In [7]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

In [8]:
# Define hyperparameter space
param_space = [
    Integer(50, 300, name='n_estimators'),
    Integer(2, 20, name='max_depth'),
    Integer(2, 20, name='min_samples_split'),
    Integer(1, 20, name='min_samples_leaf'),
    Categorical(['sqrt', 'log2', None], name='max_features'),
    Categorical([True, False], name='bootstrap')
]

In [9]:
# Define the objective function
@use_named_args(param_space)
def objective(**params):
    model = RandomForestClassifier(
        random_state=42,
        n_jobs=-1,
        **params
    )
    cv_scores = cross_val_score(
        model, X_train_full, y_train_full, cv=5, scoring='accuracy', n_jobs=-1
    )
    score = -np.mean(cv_scores)
    return score

In [ ]:
# Run the optimizer
#gp_minimize is the combination of surrogate and Acquisition function.
res = gp_minimize(
    func=objective,
    dimensions=param_space,
    n_calls=10,
    verbose=True
)

Iteration No: 1 started. Evaluating function at random point.


Iteration No: 1 ended. Evaluation done at random point.
Time taken: 3.6733
Function value obtained: -0.7752
Current minimum: -0.7752
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 3.4996
Function value obtained: -0.7688
Current minimum: -0.7752
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 3.4700
Function value obtained: -0.7622
Current minimum: -0.7752
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 2.3152
Function value obtained: -0.7198
Current minimum: -0.7752
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.4825
Function value obtained: -0.7688
Current minimum: -0.7752
Iteration No: 6 started. Evaluating function at random point.
Iteration No: 6 ended. Ev

In [11]:
res

          fun: -0.775223244035719
            x: [np.int64(63), np.int64(18), np.int64(16), np.int64(6), 'log2', True]
    func_vals: [-7.752e-01 -7.688e-01 -7.622e-01 -7.198e-01 -7.688e-01
                -7.687e-01 -7.687e-01 -7.590e-01 -7.395e-01 -7.248e-01]
      x_iters: [[np.int64(63), np.int64(18), np.int64(16), np.int64(6), 'log2', True], [np.int64(202), np.int64(17), np.int64(11), np.int64(18), 'log2', True], [np.int64(204), np.int64(14), np.int64(13), np.int64(4), 'log2', False], [np.int64(220), np.int64(10), np.int64(11), np.int64(9), None, False], [np.int64(110), np.int64(17), np.int64(3), np.int64(19), 'sqrt', True], [np.int64(278), np.int64(10), np.int64(4), np.int64(2), 'sqrt', True], [np.int64(69), np.int64(19), np.int64(18), np.int64(7), 'sqrt', True], [np.int64(209), np.int64(16), np.int64(17), np.int64(5), 'log2', False], [np.int64(127), np.int64(7), np.int64(19), np.int64(11), None, False], [np.int64(215), np.int64(13), np.int64(14), np.int64(14), None, False]]
    

In [12]:
# Get the best hyperparameters
best_params = {
    'n_estimators': res.x[0],
    'max_depth': res.x[1],
    'min_samples_split': res.x[2],
    'min_samples_leaf': res.x[3],
    'max_features': res.x[4],
    'bootstrap': res.x[5]
}

print("Best Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")



Best Hyperparameters:
n_estimators: 63
max_depth: 18
min_samples_split: 16
min_samples_leaf: 6
max_features: log2
bootstrap: True


In [14]:
# Evaluate the tuned model
best_model = RandomForestClassifier(
    random_state=42,
    n_jobs=-1,
    **best_params
)
best_model.fit(X_train_full, y_train_full)
y_pred = best_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\nTest Accuracy: {acc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))



Test Accuracy: 0.7273

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.84      0.80       100
           1       0.64      0.52      0.57        54

    accuracy                           0.73       154
   macro avg       0.70      0.68      0.69       154
weighted avg       0.72      0.73      0.72       154

Confusion Matrix:
[[84 16]
 [26 28]]
